In [ ]:
<path d="M18.4 14.6 12 8.3l-6.4 6.3.8.8L12 9.7l5.6 5.7z"></path>
<path d="m18 9.28-6.35 6.35-6.37-6.35.72-.71 5.64 5.65 5.65-5.65z"></path>

In [44]:
#link = "https://www.youtube.com/watch?v=QRKFLqMa5gw"
#link = "https://www.youtube.com/watch?v=OpP_WNGgY1k"
link = "https://www.youtube.com/watch?v=qz2AVif8FWY"

In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from time import sleep
import os
import pandas as pd

cwd = os.getcwd()

PATH = cwd + "\ChromeDriver\chromedriver.exe"

driver = webdriver.Chrome(service=ChromeService(PATH))
driver.get(link)

prev_height = 0
scroll_counter = 100

# Scroll Down all coments

while(True):
    height = driver.execute_script('return document.documentElement.scrollHeight')
    driver.execute_script("window.scrollTo(0, " + str(height) + ");")
    
    if prev_height == height:
        scroll_counter-=1
        if scroll_counter == 0:
            break
    else:
        scroll_counter = 100
        
    prev_height = height

In [47]:
def get_content(comment):
    
    content_text = ''
    
    content = comment.find_element("xpath", ".//yt-formatted-string[@id = 'content-text']")
    #contents = comment.find_elements("xpath", "//*[@id='content-text']/span")
    
    try:
        for sub_content in content.find_element("xpath", "./span"):
            content_text = content_text + sub_content.text + " "
    except:
        content_text += content.text       
    
    return content_text

In [48]:
df = pd.DataFrame(columns = ["Comments"])

comments_XPATH = "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-comments/ytd-item-section-renderer/div[3]"
comments_section = driver.find_element("xpath", comments_XPATH)
comments = comments_section.find_elements("xpath",'./ytd-comment-thread-renderer')
print(len(comments))

# Press all view replies button to comments
'''for comment in comments:
    try:
        see_replies = comment.find_element("xpath", ".//path[@d = 'm18 9.28-6.35 6.35-6.37-6.35.72-.71 5.64 5.65 5.65-5.65z']")
        see_replies.click()
    except:
        continue'''
for comment in comments:
    try:
        see_replies = comment.find_element("xpath", './/ytd-button-renderer[@id="more-replies"]')
        see_replies.click()
        print("was there")
    except:
        print("was not there")

        
# This is not the same as previous comments
comments = comments_section.find_elements("xpath",'.//ytd-comment-thread-renderer')
#replies = comments_section.find_elements("xpath", ".//ytd-comment-renderer")

for comment in comments:
    
    content = get_content(comment)
    df.loc[len(df)] = content
    
    try:
        replies_section = comment.find_element("xpath", ".//ytd-comment-replies-renderer")
        replies = replies_section.find_elements("xpath", ".//ytd-comment-renderer")
        for reply in replies:
            content = get_content(reply)
            df.loc[len(df)] = content
    except:
        continue
    
print(df)

13
was not there
was not there
was not there
was not there
was there
was not there
was not there
was not there
was there
was not there
was not there
was not there
was not there


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//ytd-comment-replies-renderer"}
  (Session info: chrome=114.0.5735.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x008DA813+48355]
	(No symbol) [0x0086C4B1]
	(No symbol) [0x00775358]
	(No symbol) [0x007A09A5]
	(No symbol) [0x007A0B3B]
	(No symbol) [0x00799AE1]
	(No symbol) [0x007BA784]
	(No symbol) [0x00799A36]
	(No symbol) [0x007BAA94]
	(No symbol) [0x007CC922]
	(No symbol) [0x007BA536]
	(No symbol) [0x007982DC]
	(No symbol) [0x007993DD]
	GetHandleVerifier [0x00B3AABD+2539405]
	GetHandleVerifier [0x00B7A78F+2800735]
	GetHandleVerifier [0x00B7456C+2775612]
	GetHandleVerifier [0x009651E0+616112]
	(No symbol) [0x00875F8C]
	(No symbol) [0x00872328]
	(No symbol) [0x0087240B]
	(No symbol) [0x00864FF7]
	BaseThreadInitThunk [0x760900C9+25]
	

In [34]:
df['Comments'].loc[1]

'Este fue un partidazo de mi Madrid. Me acuerdo verlo en vivo , me alegré muchísimo. Y también recuerdo que fue el último Derby en el Vicente Calderón'

In [7]:
height = driver.execute_script('return document.documentElement.scrollHeight')
driver.execute_script("window.scrollTo(0, " + str(height) + ");")